# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 21 2022 8:30AM,246877,10,SNAP920221,"Snap Medical Industries, LLC",Released
1,Sep 21 2022 8:30AM,246877,10,SNAP920222,"Snap Medical Industries, LLC",Released
2,Sep 21 2022 8:30AM,246877,10,SNAP920223,"Snap Medical Industries, LLC",Released
3,Sep 21 2022 8:30AM,246884,10,SONSB0001945,"Nextsource Biotechnology, LLC",Released
4,Sep 21 2022 8:12AM,246899,10,Eme-104354,Emerginnova,Released
5,Sep 21 2022 8:12AM,246899,10,Enova-10669,Emerginnova,Released
6,Sep 21 2022 8:12AM,246899,10,Enova-10670,Emerginnova,Released
7,Sep 21 2022 8:12AM,246899,10,Enova-10671,Emerginnova,Released
8,Sep 21 2022 8:12AM,246899,10,Enova-10672,Emerginnova,Released
9,Sep 21 2022 8:12AM,246899,10,Eme-104365,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,246884,Released,1
33,246896,Released,4
34,246897,Executing,1
35,246898,Released,16
36,246899,Released,9


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246884,NaN,NaN,1.0
246896,NaN,NaN,4.0
246897,NaN,1.0,NaN
246898,NaN,NaN,16.0
246899,NaN,NaN,9.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246711,29.0,0.0,3.0
246717,0.0,0.0,1.0
246727,0.0,0.0,1.0
246789,0.0,0.0,1.0
246811,41.0,0.0,8.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246711,29,0,3
246717,0,0,1
246727,0,0,1
246789,0,0,1
246811,41,0,8


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246711,29,0,3
1,246717,0,0,1
2,246727,0,0,1
3,246789,0,0,1
4,246811,41,0,8


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246711,29,,3
1,246717,,,1
2,246727,,,1
3,246789,,,1
4,246811,41,,8


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 21 2022 8:30AM,246877,10,"Snap Medical Industries, LLC"
3,Sep 21 2022 8:30AM,246884,10,"Nextsource Biotechnology, LLC"
4,Sep 21 2022 8:12AM,246899,10,Emerginnova
13,Sep 21 2022 8:12AM,246898,10,ISDIN Corporation
29,Sep 21 2022 8:00AM,246897,22,"NBTY Global, Inc."
30,Sep 21 2022 7:44AM,246896,19,"AdvaGen Pharma, Ltd"
34,Sep 20 2022 4:04PM,246878,21,"NBTY Global, Inc."
35,Sep 20 2022 4:03PM,246727,21,"NBTY Global, Inc."
36,Sep 20 2022 3:39PM,246717,21,"NBTY Global, Inc."
37,Sep 20 2022 3:12PM,246872,12,Doctors Scientific Organica


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 21 2022 8:30AM,246877,10,"Snap Medical Industries, LLC",,,3
1,Sep 21 2022 8:30AM,246884,10,"Nextsource Biotechnology, LLC",,,1
2,Sep 21 2022 8:12AM,246899,10,Emerginnova,,,9
3,Sep 21 2022 8:12AM,246898,10,ISDIN Corporation,,,16
4,Sep 21 2022 8:00AM,246897,22,"NBTY Global, Inc.",,1,
5,Sep 21 2022 7:44AM,246896,19,"AdvaGen Pharma, Ltd",,,4
6,Sep 20 2022 4:04PM,246878,21,"NBTY Global, Inc.",,,1
7,Sep 20 2022 4:03PM,246727,21,"NBTY Global, Inc.",,,1
8,Sep 20 2022 3:39PM,246717,21,"NBTY Global, Inc.",,,1
9,Sep 20 2022 3:12PM,246872,12,Doctors Scientific Organica,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 8:30AM,246877,10,"Snap Medical Industries, LLC",3,,
1,Sep 21 2022 8:30AM,246884,10,"Nextsource Biotechnology, LLC",1,,
2,Sep 21 2022 8:12AM,246899,10,Emerginnova,9,,
3,Sep 21 2022 8:12AM,246898,10,ISDIN Corporation,16,,
4,Sep 21 2022 8:00AM,246897,22,"NBTY Global, Inc.",,1,
5,Sep 21 2022 7:44AM,246896,19,"AdvaGen Pharma, Ltd",4,,
6,Sep 20 2022 4:04PM,246878,21,"NBTY Global, Inc.",1,,
7,Sep 20 2022 4:03PM,246727,21,"NBTY Global, Inc.",1,,
8,Sep 20 2022 3:39PM,246717,21,"NBTY Global, Inc.",1,,
9,Sep 20 2022 3:12PM,246872,12,Doctors Scientific Organica,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 8:30AM,246877,10,"Snap Medical Industries, LLC",3,,
1,Sep 21 2022 8:30AM,246884,10,"Nextsource Biotechnology, LLC",1,,
2,Sep 21 2022 8:12AM,246899,10,Emerginnova,9,,
3,Sep 21 2022 8:12AM,246898,10,ISDIN Corporation,16,,
4,Sep 21 2022 8:00AM,246897,22,"NBTY Global, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 8:30AM,246877,10,"Snap Medical Industries, LLC",3.0,NaN,NaN
1,Sep 21 2022 8:30AM,246884,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
2,Sep 21 2022 8:12AM,246899,10,Emerginnova,9.0,NaN,NaN
3,Sep 21 2022 8:12AM,246898,10,ISDIN Corporation,16.0,NaN,NaN
4,Sep 21 2022 8:00AM,246897,22,"NBTY Global, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 8:30AM,246877,10,"Snap Medical Industries, LLC",3.0,0.0,0.0
1,Sep 21 2022 8:30AM,246884,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
2,Sep 21 2022 8:12AM,246899,10,Emerginnova,9.0,0.0,0.0
3,Sep 21 2022 8:12AM,246898,10,ISDIN Corporation,16.0,0.0,0.0
4,Sep 21 2022 8:00AM,246897,22,"NBTY Global, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2715531,39.0,10.0,20.0
12,246872,1.0,0.0,0.0
15,493522,11.0,0.0,70.0
16,1234241,10.0,3.0,8.0
19,246896,4.0,0.0,0.0
20,246828,37.0,25.0,15.0
21,740322,3.0,0.0,0.0
22,493746,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2715531,39.0,10.0,20.0
1,12,246872,1.0,0.0,0.0
2,15,493522,11.0,0.0,70.0
3,16,1234241,10.0,3.0,8.0
4,19,246896,4.0,0.0,0.0
5,20,246828,37.0,25.0,15.0
6,21,740322,3.0,0.0,0.0
7,22,493746,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,39.0,10.0,20.0
1,12,1.0,0.0,0.0
2,15,11.0,0.0,70.0
3,16,10.0,3.0,8.0
4,19,4.0,0.0,0.0
5,20,37.0,25.0,15.0
6,21,3.0,0.0,0.0
7,22,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,39.0
1,12,Released,1.0
2,15,Released,11.0
3,16,Released,10.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,20.0,0.0,70.0,8.0,0.0,15.0,0.0,0.0
Executing,10.0,0.0,0.0,3.0,0.0,25.0,0.0,1.0
Released,39.0,1.0,11.0,10.0,4.0,37.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,20.0,0.0,70.0,8.0,0.0,15.0,0.0,0.0
1,Executing,10.0,0.0,0.0,3.0,0.0,25.0,0.0,1.0
2,Released,39.0,1.0,11.0,10.0,4.0,37.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,20.0,0.0,70.0,8.0,0.0,15.0,0.0,0.0
1,Executing,10.0,0.0,0.0,3.0,0.0,25.0,0.0,1.0
2,Released,39.0,1.0,11.0,10.0,4.0,37.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()